In [3]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
# import soundfile as sf
from scipy.fft import rfft,rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')

In [4]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [5]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [6]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [7]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [8]:
def features_extractor(file):
    features=[]
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast') 
    mfcc=mfcc_feature_extractor(audio,sampleRate)
    contrast = contrast_feature_extractor(audio,sampleRate)
    tonnetz = tonnetz_feature_extractor(audio,sampleRate)
    chroma = chroma_feature_extractor(audio,sampleRate)

    features.append([mfcc,contrast,tonnetz,chroma])
    features[0] = np.concatenate((features[0][0],features[0][1],features[0][2],features[0][3]))
    return features

In [9]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        contrast = contrast_feature_extractor(audio,sampleRate)
        tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        chroma = chroma_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,contrast,tonnetz,chroma,finalClassLabels])

In [10]:
dict = {"in group":
        ["C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/backend/files/Anwar",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/backend/files/Aya",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/backend/files/Ehab",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/backend/files/Zeyad",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/anwar",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/aya",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/ehab",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/zeyad",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/words/Anwar",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/words/Aya",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/words/Ehab",
         "C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/words/Zeyad"],
                "out of group":
                    ["C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/backend/outGroup"]}

    


In [11]:
extractedFeatures = []
for i in dict.keys():
    for x in dict[i]:
        add_features(extractedFeatures,x,i)

In [12]:
data=pd.DataFrame(extractedFeatures,columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.head()

(2247, 5)


,mfcc,contrast,tonnetz,chroma,class
0,"[-506.52463, 61.64891, 8.294457, 6.1529727, 16...","[18.225099789410645, 14.073347582034515, 14.92...","[-0.026156166403453775, -0.007522828056776607,...","[0.7895138, 0.7891665, 0.7331944, 0.69265306, ...",in group
1,"[-484.83054, 57.63201, 19.381887, 17.619091, 1...","[20.11971653181864, 14.4055109778802, 15.14931...","[0.020898464482939905, 0.02311273234743904, 0....","[0.7711498, 0.77708644, 0.7428681, 0.6990656, ...",in group
2,"[-375.71805, 95.867195, -0.6657963, 31.796244,...","[18.940401074917414, 15.716624572258736, 17.18...","[0.0030600953263598307, 0.004340534464086048, ...","[0.62990934, 0.66626626, 0.70043945, 0.7260682...",in group
3,"[-351.94073, 53.247982, 12.9672785, 10.35929, ...","[18.990867307267536, 14.260342060954631, 16.18...","[-0.013442975463150214, -0.03099029914648438, ...","[0.7548545, 0.85651636, 0.81929094, 0.6672034,...",in group
4,"[-442.27194, 81.52423, 17.646692, 13.47512, 12...","[20.260033821862628, 13.37880827112114, 15.661...","[-0.021241162685261376, -0.021280018761884012,...","[0.665574, 0.78218377, 0.7599841, 0.68303883, ...",in group


In [13]:
features=data.iloc[:,0:-1]
target=data['class']
features.head()

,mfcc,contrast,tonnetz,chroma
0,"[-506.52463, 61.64891, 8.294457, 6.1529727, 16...","[18.225099789410645, 14.073347582034515, 14.92...","[-0.026156166403453775, -0.007522828056776607,...","[0.7895138, 0.7891665, 0.7331944, 0.69265306, ..."
1,"[-484.83054, 57.63201, 19.381887, 17.619091, 1...","[20.11971653181864, 14.4055109778802, 15.14931...","[0.020898464482939905, 0.02311273234743904, 0....","[0.7711498, 0.77708644, 0.7428681, 0.6990656, ..."
2,"[-375.71805, 95.867195, -0.6657963, 31.796244,...","[18.940401074917414, 15.716624572258736, 17.18...","[0.0030600953263598307, 0.004340534464086048, ...","[0.62990934, 0.66626626, 0.70043945, 0.7260682..."
3,"[-351.94073, 53.247982, 12.9672785, 10.35929, ...","[18.990867307267536, 14.260342060954631, 16.18...","[-0.013442975463150214, -0.03099029914648438, ...","[0.7548545, 0.85651636, 0.81929094, 0.6672034,..."
4,"[-442.27194, 81.52423, 17.646692, 13.47512, 12...","[20.260033821862628, 13.37880827112114, 15.661...","[-0.021241162685261376, -0.021280018761884012,...","[0.665574, 0.78218377, 0.7599841, 0.68303883, ..."


In [14]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [15]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=0)

In [25]:
classifier= SVC(kernel='linear', decision_function_shape="ovo")
classifier.fit(xTrain,yTrain)

SVC(decision_function_shape='ovo', kernel='linear')

In [18]:
preds = classifier.predict(xTest)

In [19]:
def prediction(pred):
    if pred==0:
        print("in group")
    elif pred==1:
        print("out of group")


In [20]:
print(classification_report(yTest,preds))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       290
           1       0.98      0.93      0.95       272

    accuracy                           0.96       562
   macro avg       0.96      0.96      0.96       562
weighted avg       0.96      0.96      0.96       562



In [22]:
testFeatures = features_extractor("C:/Users/Anwar/Desktop/SBME 2024/YEAR 3 (2022-2023)/DSP/Tasks/Task 3/DSP_Task3/processing/words/Aya/voice01100741.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


in group


In [23]:
pickle_out = open("ingroup_model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [26]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from sklearn.linear_model import LinearRegression
# Sequential Forward Selection(sfs)
sfs = SFS(SVC(kernel='linear', decision_function_shape="ovo"),
          k_features=2,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

In [28]:
features = pd.DataFrame(features)

In [29]:

sfs.fit(features, target)
sfs.k_feature_names_

(4, 45)

In [39]:
xTrain = pd.DataFrame(xTrain)
yTrain = pd.DataFrame(yTrain)
xTrain

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,-266.738708,103.663437,4.573472,9.041640,1.614694,1.396647,-0.366143,2.718201,-15.133023,7.939022,...,0.592345,0.557635,0.550570,0.579080,0.587402,0.608454,0.681707,0.665095,0.649609,0.642544
1,-448.730499,128.073349,5.418815,17.657125,8.967704,21.781902,-9.745255,6.164220,-4.046611,-1.489009,...,0.512589,0.468016,0.495885,0.496994,0.564369,0.605995,0.568163,0.564449,0.481528,0.470946
2,-132.712402,124.452629,-57.859756,21.284811,-42.651840,12.879964,-33.533676,-12.255640,-32.373508,-10.699275,...,0.659431,0.646030,0.656923,0.688460,0.675429,0.721130,0.809615,0.864457,0.729332,0.585895
3,-477.843170,83.065918,23.156742,10.397629,10.948236,3.133870,4.281215,8.819442,-5.246666,-8.278208,...,0.724060,0.646809,0.633086,0.608432,0.558200,0.619973,0.699888,0.731496,0.753554,0.716039
4,-362.739746,88.982140,13.191752,-10.316602,-4.570636,13.831303,14.777443,5.533417,-12.949487,-5.941306,...,0.504630,0.515603,0.582246,0.713369,0.724475,0.597735,0.443942,0.369199,0.328193,0.371599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,-360.629028,85.946045,20.472349,14.858758,10.075060,9.663091,3.764751,4.749082,-9.372436,-7.246763,...,0.490178,0.481995,0.494331,0.580687,0.596123,0.533046,0.587853,0.619840,0.598977,0.635037
1681,-402.067444,66.127922,29.439125,23.981352,1.161905,1.687719,5.675429,-2.388285,-4.676038,-2.982301,...,0.643399,0.555903,0.609044,0.582295,0.589470,0.655952,0.708748,0.622503,0.554623,0.579503
1682,-408.519897,133.835327,11.104596,12.191189,17.834549,4.502982,0.613786,4.020524,-7.504054,0.020026,...,0.621752,0.674013,0.733350,0.665058,0.666229,0.576183,0.576828,0.623941,0.590552,0.597102
1683,-374.894928,110.480598,18.530649,21.173965,0.448625,1.714309,15.304652,-4.517304,-6.061135,-0.784284,...,0.407298,0.416184,0.403937,0.452619,0.652298,0.751744,0.602573,0.501949,0.500472,0.495754


In [48]:
from matplotlib.colors import ListedColormap  
x_set, y_set = xTrain, yTrain  
x1, x2 = np.meshgrid(np.arange(start = x_set[:, 4].min() - 1, stop = x_set[:, 4].max() + 1, step  =0.01),  
np.arange(start = x_set[:, 45].min() - 1, stop = x_set[:, 45].max() + 1, step = 0.01)) 
# print(x1) 
plt.contourf(x1, x2, classifier.predict(np.array([x1.ravel(), x2.ravel()]).T).reshape(x1.shape),  
alpha = 0.75, cmap = ListedColormap(('red', 'green')))  
plt.xlim(x1.min(), x1.max())  
plt.ylim(x2.min(), x2.max())  
for i, j in enumerate(np.unique(y_set)):  
    plt.scatter(x_set[y_set == j, 0], x_set[y_set == j, 1],  
        c = ListedColormap(('red', 'green'))(i), label = j)  
plt.title('SVM classifier (Training set)')  
plt.xlabel('Age')  
plt.ylabel('Estimated Salary')  
plt.legend()  
plt.show()

InvalidIndexError: (slice(None, None, None), 4)